In [1]:
# 1. 판다스 도구 불러오기
import pandas as pd
import os
import numpy as np

# --- 숫자 칸 합치는 함수 ---
def combine_split_numbers(row):
    # try...except를 제거했으므로, 여기서 오류 발생 시 프로그램 중단됨
    # 임시 컬럼 이름 ('칸_4', '칸_5') 대신 실제 사용할 이름으로 수정
    try:
        a = pd.to_numeric(row.get('미분양_A', np.nan), errors='coerce')
        b = pd.to_numeric(row.get('미분양_B', np.nan), errors='coerce')
        if pd.notna(b) and pd.notna(a): return int(a * 1000 + b)
        elif pd.notna(a): return int(a)
        elif pd.notna(b): return int(b)
        else: return np.nan
    except: return np.nan # 아주 간단한 오류 처리

# --- 보물 지도 파일 두 개 위치 알려주기 ---
file_path = r'C:\RAG_COMMANDER\src\data\unsold_units\미분양주택현황보고_시·군·구별 미분양현황 (2017.08 ~2021.08).csv'
file_path2 = r'C:\RAG_COMMANDER\src\data\unsold_units\미분양주택현황보고_시·군·구별 미분양현황 (2021.08 ~ 202508).csv' # 두 번째 파일 경로

# 두 지도를 담을 빈 가방(리스트) 준비
all_maps = []

# --- 첫 번째 지도 열기 ---
print(f"'{os.path.basename(file_path)}' 지도를 열어볼게!")
# 컬럼 이름을 제대로 지정: 연도월, 시도, 시군구, 미분양_A, 미분양_B
col_names = ['연도월', '시도', '시군구', '미분양_A', '미분양_B']
treasure_map1 = pd.read_csv(file_path, encoding='cp949', skiprows=1, header=None, names=col_names, on_bad_lines='skip')
print(" -> 첫 번째 지도 열기 완료!")
all_maps.append(treasure_map1)

# --- 두 번째 지도 열기 ---
print(f"'{os.path.basename(file_path2)}' 지도를 열어볼게!")
treasure_map2 = pd.read_csv(file_path2, encoding='cp949', skiprows=1, header=None, names=col_names, on_bad_lines='skip')
print(" -> 두 번째 지도 열기 완료!")
all_maps.append(treasure_map2)

# --- 가방에 있는 지도들을 하나로 합치고 정리하기 ---
print("\n--- 지도 합치기! ---")
combined_map = pd.concat(all_maps, ignore_index=True)

print("--- 데이터 정리 중... ---")
# 숫자 합치기
combined_map['미분양'] = combined_map.apply(combine_split_numbers, axis=1)
combined_map.dropna(subset=['미분양'], inplace=True)
combined_map['미분양'] = combined_map['미분양'].astype(int)

# 날짜 정리 (YYYY.MM 형식 가정)
date_str = combined_map['연도월'].astype(str).str.strip()
# '.' 또는 '-' 둘 다 처리 가능하도록 수정
ym_split = date_str.str.split('[.-]', expand=True) # 정규식 사용
if ym_split.shape[1] >= 2:
    combined_map['연도'] = pd.to_numeric(ym_split[0], errors='coerce')
    combined_map['월'] = pd.to_numeric(ym_split[1], errors='coerce')
    combined_map.dropna(subset=['연도', '월'], inplace=True)
    combined_map['연도'] = combined_map['연도'].astype(int)
    combined_map['월'] = combined_map['월'].astype(int)
else:
    print("경고: 날짜 형식이 예상과 달라 연도/월 분리에 실패했습니다.")
    # 연도, 월 컬럼이 없으면 이후 필터링 불가 -> 종료 또는 다른 처리 필요
    combined_map = pd.DataFrame() # 빈 데이터프레임으로 만듦


# 글자 정리 (시도, 시군구)
combined_map['시도'] = combined_map['시도'].astype(str).str.strip()
combined_map['시군구'] = combined_map['시군구'].astype(str).str.strip()

# 필요한 최종 칸만 남기기
if not combined_map.empty:
    final_data = combined_map[['연도', '월', '시도', '시군구', '미분양']].copy()
    final_data = final_data.drop_duplicates()
    final_data = final_data.sort_values(by=['연도', '월', '시도', '시군구'])

    # --- ⬇️⬇️⬇️ 서울시 데이터만 골라내기! ⬇️⬇️⬇️ ---
    print("\n--- 서울시 데이터만 고르는 중... ---")
    seoul_only_data = final_data[final_data['시도'] == '서울'].copy() # '시도' 칸이 '서울'인 것만!

    # --- 골라낸 서울 데이터 전체 출력 ---
    if seoul_only_data.empty:
        print("\n❌ 합쳐진 데이터에 '서울' 데이터가 하나도 없어...")
        print("   '시도' 칸 이름이 '서울'이 맞는지, 데이터가 제대로 읽혔는지 확인해봐!")
        print("\n--- 참고: 합쳐진 데이터의 '시도' 종류 ---")
        print(final_data['시도'].unique()) # 어떤 시도 이름들이 있는지 확인
    else:
        print("✅ 서울 데이터만 골라내기 성공!")
        print("\n--- 서울시 미분양 데이터 전체 보기 (과거부터) ---")
        # 표 전체를 깔끔하게 출력 (생략 없이)
        with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.width', 1000):
            print(seoul_only_data)

else:
    print("\n데이터를 제대로 정리하지 못해서 서울 데이터를 고를 수 없어...")
                                                                                                                         
print("\n--- 작업 끝! ---")

'미분양주택현황보고_시·군·구별 미분양현황 (2017.08 ~2021.08).csv' 지도를 열어볼게!
 -> 첫 번째 지도 열기 완료!
'미분양주택현황보고_시·군·구별 미분양현황 (2021.08 ~ 202508).csv' 지도를 열어볼게!
 -> 두 번째 지도 열기 완료!

--- 지도 합치기! ---
--- 데이터 정리 중... ---

--- 서울시 데이터만 고르는 중... ---
✅ 서울 데이터만 골라내기 성공!

--- 서울시 미분양 데이터 전체 보기 (과거부터) ---
         연도   월  시도   시군구   미분양
23     2017   8  서울   강남구     0
25     2017   8  서울   강동구     0
9      2017   8  서울   강북구     0
16     2017   8  서울   강서구     0
0      2017   8  서울     계    39
21     2017   8  서울   관악구     0
5      2017   8  서울   광진구     0
17     2017   8  서울   구로구     0
18     2017   8  서울   금천구     0
11     2017   8  서울   노원구     0
10     2017   8  서울   도봉구     2
6      2017   8  서울  동대문구     0
20     2017   8  서울   동작구     0
14     2017   8  서울   마포구     0
13     2017   8  서울  서대문구     0
22     2017   8  서울   서초구    16
4      2017   8  서울   성동구     1
8      2017   8  서울   성북구     8
24     2017   8  서울   송파구     8
15     2017   8  서울   양천구     3
19     2017   8  서울  영등포구     0
3      2017   8  서울   용산구

In [2]:
seoul_only_data = seoul_only_data.sort_index(ascending=False)
seoul_only_data

,연도,월,시도,시군구,미분양
23887,2025,8,서울,강동구,382
23886,2025,8,서울,송파구,0
23885,2025,8,서울,강남구,0
23884,2025,8,서울,서초구,0
23883,2025,8,서울,관악구,4
...,...,...,...,...,...
4,2017,8,서울,성동구,1
3,2017,8,서울,용산구,0
2,2017,8,서울,중구,0
1,2017,8,서울,종로구,1


In [3]:
# CSV로 저장
seoul_only_data.to_csv('미분양_데이터.csv', encoding='utf-8-sig')
print("✅ CSV 파일 저장 완료: 미분양_데이터.csv")

✅ CSV 파일 저장 완료: 미분양_데이터.csv
